Chapter 13: A Gnuplot Miscellany
================================

This chapter contains examples that don’t have an obvious place in any
of the other chapters, along with some elaboration of previous scripts
and some further examples of things that you can do with gnuplot.

Our first example shows one way to make a bar graph that appears to be
three-dimensional. While it is, in general, a bad idea to represent 2D
data using 3D perspective, and something that is almost never used in
serious or technical reporting, the dressing up in 3D of such things as
bar charts is popular in journalistic or advertising contexts. You may
find yourself called upon to do such a thing, and wonder if you can
cajole gnuplot into playing along. Here is one way to do it, based
partly on
[ideas](http://gnuplot-tricks.blogspot.com/2009/06/3d-bargraphs-in-gnuplot-one-more-time.html)
from the Gnuplot Tricks website. We can’t use the built-in
[histogram](#bar-charts) style for this; instead, since we intend to
make 3D perspective shapes, we’ll leverage gnuplot’s `splot` command.

The latest version of gnuplot adds [true 3D box plots](#d-box-plots), so
this technique is probably no longer necessary. However, it might be
instructive, so we’ll leave it here.

3D Bars
-------

In this script, we create each 3D bar from three different rectangles;
the 3D effect is completed by using [ambient lighting](#lighting). The
data values are stored in the [array](#arrays) “DD”; the width of the
bars is controlled by “w”, and the space between them by “p”. You would
probably want to add labels, and perhaps part of the bounding box. This
script can easily be turned into a command-line tool that accepts a list
of data as an input parameter, similarly to how we created [pie-chart
tool](#pietool).

In [ ]:
unset key; unset colorbox
unset border; unset tics
set pm3d lighting primary .5
s = 0; w = .9; p = 1.4
set pal def (0 "blue", 1 "blue")
set multiplot
set parametric; set isosample 2, 2
set view 65, 30
array DD[5] = [.2, .5, 1., .7, .6]
set xr [0 : 1.5*|DD|]; set yr [0 : |DD|]; set zr [0:1]
do for [i=1:|DD|] {
  D = DD[i]
  set urange [0:w]; set vrange [0:w]
  splot u+s, v, D w pm3d
  set urange [0:w]; set vrange [0:D]
  splot u+s, 0, v w pm3d
  splot w+s, u, v w pm3d
  s = s + p
}


Plotting with Pictures
----------------------

Gnuplot can place pictures on a plot along with, or instead of, its
usual points, lines, and objects. It can understand the common image
formats, but the details depend on how your version was compiled. For a
list of the image type that your gnuplot believes that it can handle,
give it the command `show datafile binary filetypes`. If you
nevertheless get a message that gnuplot can’t read an image file that is
on the list, you may need to recompile it, ensuring that the libgd-dev
(on Linux) package has been installed. Here is the basic command for
placing an image on the plot. The image, in the file “plane.png”, which
you can download from the publisher’s website, has pixel dimensions 640
× 533; you can see how gnuplot plots the image as an array of pixels,
with the axes labeled appropriately. The keyword `rgbalpha` is
\\index{rgbalpha}{ used for images with transparency, such as this one;
use `rgbimage` for JPEGs, PNGs without transparency, etc. The clause
`filetype = auto` tells gnuplot to guess the image format from the
filename extension, but if your files have nonstandard names you can be
more specific. The angle unit can be specified in degrees (`rot 45deg`),
radians with a bare number (as we do here), or radians in units of π
(`0.5pi`). The keywords `dx` and `dy` give scaling factors for the
image. Notice how the image transparency is preserved in the overlapping
regions. The [animation techniques](#animations) we learned before could
easily be combined with image plotting to create sprite animations
within gnuplot.

In [ ]:
unset key
plot  "plane.png" binary filetype=auto with rgbalpha,\
   "plane.png" binary filetype=auto center = (100, 150)\
   dx = 1.5 dy = 0.25 rot = pi/2 with rgbalpha

This example script illustrates the commands for placing images with
specified location, scaling, and rotation on a graph in order to dress
up a plot with a thematic decoration. Since positioning depends on the
image’s pixel dimensions, we define two variables in the second line to
hold half the number of pixels in the x- and y-directions. The
`ang(a, b)` function calculates the angle from the slope of the data (in
the data array we’ve duplicated the last element for convenience). In
the first `plot` command we place a plane image at each data point,
rotated to indicate the direction to the following data point. We use
multiplot mode to plot the same data with a dotted line, adjusting the
axis ranges to get approximate alignment with the plane images. You can
omit `dy`, in which case it is set equal to `dx`.

In [ ]:
set multi
set tmargin at screen .9
set rmargin at screen .9
set lmargin at screen .3
set bmargin at screen .3
unset key
xph = 320; yph = 267
array d[5] = [1, 3, 2, 4, 4]
ang(a, b) = atan(b - a) - pi/2
set for  [n = 1 : |d|-1] xtics (sprintf("%d", 2013 + n) xph * n)
set for  [n = 1 : |d|-1] ytics (sprintf("%d", 100 * d[n]) yph * d[n])
plot for [n = 1 : |d|-1] "plane.png" binary filetype=auto \
center=(xph * n, yph * d[n]) dx = 0.3 rot=ang(d[n], d[n + 1]) with rgbalpha
set yr [.5 : 4.5]
set xr [.5 : 4.5]
set xlab "Year" font "Helvetica, 18"
set ylab "Plane production" font "Helvetica, 18" offset -3
plot d with lines lw 5 dt "-"

Pictures in 3D
--------------

You can place images in a 3D plot as well, in which case gnuplot will
draw the image with 3D perspective. Simply use `splot` instead of
`plot`, and provide a 3D coordinate. Rotation, by default, is around an
axis passing through the center of the image and perpendicular to the
x-y plane. For more details about other options for rotation in 3D (and
2D), try `help perpendicular` and `help rot`.

In [ ]:
unset key
set border 15
set zlabel "Altitude" rot by 90 font ",16" offset -1
set zr [0:1600]
set view 60,30
set xyplane 0
set xtics scale 0
set ytics scale 0
set xtics format ""
set ytics format ""
set grid lt 1 lc "green"
hs = 470; vs = 360.
scl(n) = 1. - n/8.
splot for [n = 0 : 3] "plane.png" binary filetype=auto\
   center=(n*.5*hs, n*vs, n*vs) dx=scl(n) rot = -n/12.pi with rgbalpha

Plotting with Characters
------------------------

In previous chapters we have learned how to make scatterplots by
plotting data using the `with points` command, choosing the point type,
size, and color. The plotting “point” can also be any single character;
as this includes Unicode and an optional font specification, there are
many possibilities. Here’s how it works (we are reusing a datafile that
we used in the [chapter](#whisker-plots) on errors and finance):

In [ ]:
unset key
plot "candles" w points pt "☺" font ",24"

Variable Pointtype
------------------

Not only the size and color, but the *type* of point in a `with points`
plot can be taken from the data itself. This is activated by using the
`var` keyword, as in the example script below. The size, type, and color
of the points are taken from the third, fourth, and fifth data columns,
respectively.

In [ ]:
unset key
plot "candles" w points ps var pt var lc var

The `sample` Keyword
--------------------

A recent and powerful addition to gnuplot is the `sample` keyword. We’ve
already learned how a command like `plot [a : b] f(x)` redefines the
`xrange` to cover the range within the brackets; the use of `sample` is
similar but much more flexible. The presence of the `sample` keyword,
highlighted in the script here, plots the function following the range
specifier over that range, without changing the `xrange` of the plot.
The second range specifier (also highlighted) plots the second function
over that range. The result a cleaner method for producing the result
that we [previously](#the-ternary-operator) handled using the ternary
operator and NaNs. Note that in the absence of the `sample` keyword, the
first range specifier would simply reset the `xrange` for the graph
(which, in this case, would exclude the second part of the `plot`
command).

In [ ]:
set xr [-20 : 20]
set label "Damping applied" at 4, -.6 font "Times"
set arrow from 3.9, -.6 to 0, 0 filled lc "grey40"
plot sample [-20 : 0] sin(x) notit,\
   [0 : 20] exp(-x/5.)*sin(x) dt ".-" lw 2 title "Decay phase"

The `sample` keyword has a couple more powers that are available for
data plots (from files or the “+” or “++” special filenames). Both
powers are used in the highlighted range specifier in this script. A
third number sets the sampling interval for that part of the `plot`
command, overriding the global `set samples` command; we use it here to
put space between the points. As in [range commands](#rangevars), you
can give a name to the sample variable (here “x”) which allows you to
use this name in the column specifiers rather than having to type “\$1”.

In [ ]:
set samp 1000
set style data lines
set style line lw 4
set xr [0 : 10]
f(x) = cos(2*x) * sin(x)
scale(x) = abs(f(x)) + 0.4
plot sample [0 : 1] "+" u 1:(f(x)) lw 3 title "Early range", [5 : 6] f(x) lw 3 title "Middle range",\
  [9 : 12] f(x) lw 3 title "Late range",\
    [x = 0 : 10 : 0.1] "+" u 1:(f(x)):(scale(x)) w points lc "grey70" ps var pt 6 notitle

Multiple, Overlapping 2D plots
------------------------------

Here is an example of a style that is sometimes used to compare a
handful of probability distributions, or similar things. Offsets in the
vertical direction are used along with transparency to keep everything
visible while keeping the plot compact.

In [ ]:
unset key
set ytics format ""
set xtics format ""
set yr [0:4]
set xr [-7 : 7]
set style function filledcurves
set style fill transparent solid .5
vs = 0.7
f1(x) = exp(-x**2/5)
f2(x) = f1(x+1)*(exp(-x**2/2) + sin(x)**2) 
f3(x) = f1(x-2)*(exp(-x**2/3 + cos(2*x)**2))
f4(x) = f1(x+1)*(exp(-x**4) + 0.2*sin(x)**2)
f5(x) = f1(x-1)*(exp(-x**4/4))
plot f1(x) fc "#990088",\
     f2(x) + vs fc "#0088cc",\
     f3(x) + 2*vs fc "#55ff66",\
     f4(x) + 3*vs fc "#ff6666",\
     f5(x-1) + 4*vs fc "#bbbbdd"

Fence Plots
-----------

Another way to display a set of 2D curves is to spread them out in a 3D
space to make what is sometimes called a *fence plot*. Gnuplot makes
this convenient with its `zerrorfill` style. This style allows you to
draw a curve in 3D space and surround it with a filled region, defined
either by a pair of functions or data columns, both above (higher z) and
below (lower z) the curve. The filled region can serve to represent the
error in a set of measurements defined by the curve (hence the name
`zerrorfill`), but can be drafted to represent anything you want. Note
that although this is a `splot` style, it apparently can’t be used to
add an error fill to a surface. There are two versions. The one we show
here uses five columns, for x, y, z, zmin, zmax; there is a less general
version that uses a zdelta in place of the separate zmin and zmax. You
can set separate styles for the curve and the filled region. Here is a
simple example that shows what it does:

In [ ]:
unset key
set grid ztics
set grid ytics
set grid xtics lt 3
set border 4095
set view 60, 170
set ytics 20
splot "+" u 1:($1**2):(1):(0.75):(1.25)\
   with zerrorfill fc "blue" lc "red" lw 2

As promised in the title of this section, we now show how to use the
`zerrorfill` style to create a fence plot, using the same set of
functions as we used above. The trick is to use `zmax` for the set of
functions or data lines you wish to plot, while setting `z` and `zmin`
to coincide and form the bottoms of the curves. You should include the
[`depthorder` setting](#depthorder) to ensure that the plots overlap
correctly. Here, rather than specify a color for each curve, we let
gnuplot use its default sequence. Use `x` and `y` to space out the
curves, and use a transparent fill style with no border:

In [ ]:
unset key
unset xtics
unset ytics
set yr [-12 : 15]
set xr [-5 : 25]
set style fill transparent solid .7 noborder
set style data zerrorfill
set pm3d depthorder
set xyplane at 0
set view 70, 10
vs = 5
f1(x) = exp(-x**2/5)
f2(x) = f1(x+1)*(exp(-x**2/2) + sin(x)**2) 
f3(x) = f1(x-2)*(exp(-x**2/3 + cos(2*x)**2))
f4(x) = f1(x+1)*(exp(-x**4) + 0.2*sin(x)**2)
f5(x) = f1(x-1)*(exp(-x**4/4))
splot sample [u = -5 : 5] "+" u (u):(2*u):(0):(0):(f1(u)),\
             [u = -5 : 5] ""  u (u + vs):(2*u):(0):(0):(f2(u)),\
             [u = -5 : 5] ""  u (u + 2*vs):(2*u):(0):(0):(f3(u)),\
             [u = -5 : 5] ""  u (u + 3*vs):(2*u):(0):(0):(f4(u)),\
             [u = -5 : 5] ""  u (u + 4*vs):(2*u):(0):(0):(f5(u))

Mapping
-------

Gnuplot comes with a file called “world.dat” that is a rough but usable
outline of the continents of Earth. Since our planet is approximately a
sphere, plotting this file using spherical mapping gets us a serviceable
globe, especially if we plot latitude and longitude lines, as we do
here:

In [ ]:
unset key
set mapping spherical
set angles degrees
set isosamples 30
set xrange [-1:1]
set yrange [-1:1]
unset tics
unset border
set parametric
set hidden3d
set urange [-90:90]
set vrange [0:360]
splot cos(u)*cos(v),cos(u)*sin(v),sin(u) with lines lt rgb "grey80",\
      "world.dat" with lines lt 2 lw 2

Of course the real utility of the world.dat file is to be able to plot
geographic information with the continents as background for reference.
To do this, your data needs to be stored as a function of latitude and
longitude, in the format that gnuplot expects: south of the equator is
negative latitude, as is west of the Prime Meridian. In this example we
plot the world.dat file in 2D, which results in a cylindrical projection
of the globe, along with data from the included file “earthquakes.dat”,
which contains data from the United States Geological Survey on the
locations and magnitudes of all the earthquakes in the world for three
days in January of 2012. The script sets the tics to be `geographic`,
which [translates](#geographic-coordinates) the numerical coordinates
into index{geographic coordinates} more conventional notation for
plotting, and uses geographic formatting. For all the options available
in formats for latitudes and longitudes, type `help geographic`. We use
transparent fill here both to reveal the continent outlines underneath
the data and to show, with the buildup of opacity, when earthquake
events coincide. The sixth column in the file is mapped to the third
column in the `using` specifier, with a scaling factor, to control the
circle size; this is the earthquake magnitude.

In [ ]:
unset key
set xtics geographic
set ytics geographic
set xtics 60; set ytics 40
set xr [-180 : 180]
set yr [-100 : 100]
set format x "%D’%E"
set format y "%D’%N"
set grid
set title "Earthquakes Worldwide 20Jan2012 Through 22Jan2012"
set style fill transparent solid 0.4 noborder
plot "world.dat" with lines lt -1,\
      "earthquakes.dat" using 4:3:(1.5*$6) with circles fc "red"

Arrow Axes
----------

If you want to replace gnuplot’s border with something custom, you
probably want to employ [arrows](#arrows) for the purpose. Here we
replace the border box with a pair of arrows, to create a plot style
popular in the classroom. We use graph coordinates to make the arrow
specifications work independently of the axis ranges.

In [ ]:
unset key; unset border
set ytics .2
set arrow from graph 0,0 to graph 0, 1.2 filled
set arrow from graph 0,0 to graph 1.2, 0 filled
set tmargin 5
set rmargin 20
# set border 3
set tics nomirror
set grid
plot airy(x) lw 2

Colorsequence
-------------

A recently added feature in gnuplot is the ability to choose from three
options for the `colorsequence`. The `default` is the familiar
terminal-independent magenta - green - cyan - etc. sequence; the
`classic` option produces a terminal-dependent sequence, usually
beginning red - green - blue (this was the default in previous versions
of gnuplot); the `podo` option, that we illustrate in this example,
selects a sequence of eight colors that are easier for colorblind people
to tell apart (see also the [cubehelix palette](#cubehelix-palettes)).

In [ ]:
set colorsequence podo
set samp 1000
set xr [0 : 9]
unset key
plot for [i = 1 : 8] sin(1/(x-i)) + i lw 2

Colored Axes
------------

As we’ve [seen](#using-a-second-y-axis), you can use different scales
for different curves on a plot by setting up a second y- or x-axis. This
requires you to then indicate which curves go with which axes. You can
use different colors or dash patterns, along with a legend, or use text
labels, perhaps in combination with arrows, for this purpose. Another
option is illustrated in this example: you can color the tic and axis
labels, and use corresponding colors for the curves. Gnuplot uses a
single color for all the border lines, however.

In [ ]:
set xr [-pi : pi]
unset key
set ytics nomirror 
set y2tics nomirror tc lt 3
set ylab "sin(x)" font "Helvetica, 16"
set y2label "x^2" rot by 0 font "Helvetica, 16" tc lt 3
plot sin(x) lt -1, x**2 axis x1y2 lt 3

A variation of the previous idea is to use a [zeroaxis](#zeroaxis) for
one of the curves. This allows us to color the axis as well, but, sadly,
the tic marks themselves still inherit their color from the border.

In [ ]:
set border 3
unset key
set xr [-pi : pi]
set y2zeroaxis lt 3
set xtics nomirror
set ytics nomirror
set ylab "sin(x)" font "Helvetica, 16"
set label "x^2" font "Helvetica, 16" at -0.5, second 8 tc lt 3
set y2tics axis nomirror tc lt 3
plot sin(x) lt -1, x**2 axis x1y2 lt 3

In order to get the tic and axis labels, tic marks, and the border or
axis line itself to all be drawn in a specified color, we can resort to
multiplot mode, as in the example here. To get the plots to align, we
set global margins using screen coordinates.

In [ ]:
set multi
set tmargin at screen 0.9
set rmargin at screen 0.9
set lmargin at screen 0.1
set bmargin at screen 0.1
unset key
set xr [-pi : pi]
set tics nomirror
set border 8 lt 4
unset ytics
set y2tics 1
unset xtics
set y2label "x^2" rot by 0 font "Helvetica, 16" tc lt 4
plot x**2 axis x1y2 lt 4
set border 7 lt -1
unset y2label
set tics nomirror
unset y2tics
set ylab "sin(x)" font "Helvetica, 16"
plot sin(x) lt -1

Data Dependent Gridding
-----------------------

Gnuplot, by default, places the ticmarks at equally spaced intervals,
with gridlines to match. In cases where there we are plotting a moderate
number of discrete data points, it can be more useful to align the tics
and grid lines with the data, allowing the viewer to easily read off the
exact data values without having to interpolate between tic values.

In [ ]:
set samples 5
set key top left
set for [n = 1 : 4] arrow from first n, 0\
   to first n, n**2 nohead dt "-"
set for [n = 1 : 4] arrow from first 1, n**2\
   to first n, n**2 nohead dt "-"
set for [n = 1 : 5] ytics (n**2)
set xtics 1
plot [1 : 5] x**2 with linespoints pt 7 ps 2

Broken Axis
-----------

There is a style of plot that includes two disjoint ranges on the (for
example) x-axis. The break in the axis requires some type of marking to
indicate the discontinuity in scale. In this example we use multiplot
mode to plot the two sides of the graph, margins in screen coordinates
to align the graphs, and headless arrows to form short line segments to
indicate the position of the axis break. A variation of this style would
be to arrange for one of the tic marks to fall at the location of the
axis break and dispense with the arrow markings. You can alter this
example to achieve that effect by setting the `xtic` interval for the
first graph to 0.01 and to 0.05 for the second graph.

In [ ]:
set multi
t = 0.015; s = 0.03
unset key
set samp 1000
set border 3; set xtics .02; set tics nomirror
set bmargin at screen 0.1; set tmargin at screen 0.9
set rmargin at screen 0.4; set lmargin at screen 0.1
set xr [.01 : 0.05]
set arrow 1 from screen .4, .1 to screen 0.4 + t, 0.1 + s nohead lw 1.5
set arrow 2 from screen .4, .1 to screen 0.4 - t, 0.1 - s nohead lw 1.5
plot sin(1/x)
set lmargin at screen 0.43; set rmargin at screen 0.9
set xr [0.1 : 0.3]
set key; set border 1; unset ytics; set xtics 0.06
set arrow 1 from screen 0.43, .1 to screen 0.43 + t, 0.1 + s nohead lw 1.5
set arrow 2 from screen 0.43, .1 to screen 0.43 - t, 0.1 - s nohead lw 1.5
plot sin(1/x)

Jitter
------

Often, data is gathered in a way that causes one or more variables to be
“quantized,” rather than to be continuously distributed. For example,
some observation may be recorded at particular time intervals, which
would cause the observations to be clustered on the time axis. This will
cause the data points to overlap on a scatterplot, for example, which
would obscure some of the data. One way to overcome this is to add a
small displacement, random or otherwise, to the plotted data points, in
order to spread them out and reveal the obscured ones. Gnuplot has an
automated, configurable mechanism for doing just this, using the
recently added `jitter` command. We’ll show how it works using some
artificial data. First, an example that illustrates the problem (the
script uses the modulo operator `%`, which gives the remainder after
dividing by the right-hand operand: type
`help expressions operators binary` to see all of them): \\index{%}

In [ ]:
unset key
set grid lt -1
seed = rand(9)
plot sample [i = 0 : 99] "+" u (int(i)%10):(int(rand(0)*99)%10)\
    pt 6 ps 3 lw 3

The previous example plotted some random points on a grid, but you can’t
see all of them, because some points overlap. To reveal the hidden data,
use the `jitter` command (`help jitter` will get you details on all the
options, but the `spread` setting will cover the majority of cases; any
coordinate system can be used, with character as the default):

In [ ]:
set jitter spread 0.2
unset key
set grid lt -1
seed = rand(9)
plot sample [i = 0 : 99] "+" u (int(i)%10):(int(rand(0)*99)%10)\
    pt 6 ps 3 lw 3

`jitter` also works with impulse plots. Here’s a similar set of data
plotted with impulses. We use the `set offset` command to add some extra
space between the data and the border:

In [ ]:
unset key
set offset 0.3, 0.3
set grid
seed = rand(8)
plot sample [i = 0 : 20] "+" u (int(i)%10):(int(rand(0)*99))\
   with impulses lw 2

In the previous plot, most of the data is obscured due to overlap.
Adding a little jitter reveals all the data:

In [ ]:
unset key
set jitter spread 0.9
set offset 0.3, 0.3
set grid
seed = rand(8)
plot sample [i = 0 : 20] "+" u (int(i)%10):(int(rand(0)*99))\
   with impulses lw 2

Scatterplots of Dense Data Sets
-------------------------------

A plot using unconnected points, dots, or symbols in the plane of data
with separate x- and y-values specified is called a “scatterplot.” In
gnuplot these can be created using the plot style `with points`.
Scatterplots are useful to reveal several aspects of the data,
especially correlations between values on the two axes. But when there
are many data points and the data becomes dense in certain parts of the
graph, the points overlap, and you lose information about the density of
points in the overlap region. This example illustrates the phenomenon.
The “data” is arranged so that there is an increasing density as we
approach (0, 0). But, although we’re using a small `pointsize`, the
overlap almost entirely hides the nature of the data.

In [ ]:
unset key
set samp 600
set iso 600
set xr [0:5]
set yr [0:5]
e = 1.3
plot "++" u ($1 * rand(0)**e):($2 * rand(0)**e)\
  with points pt 7 lc "#990099" ps .5

We can improve on this significantly by using the plot style
`with dots`, which places a very small dot at each point (the smaller
the dot, the less overlap we’ll get):

In [ ]:
unset key
set samp 600
set iso 600
set xr [0:5]
set yr [0:5]
e = 1.3
plot "++" u ($1 * rand(0)**e):($2 * rand(0)**e) with dots

The previous example still had enough overlap to obscure some of the
data behavior. You may be able to improve the plot in these situations
by using nearly transparent, small points. Overlapping points will build
up opacity, naturally leading to smooth gradients in density that match
the data gradients. Experimenting with the alpha value and `pointsize`
will help to get an optimal visualization for any particular data set.

In [ ]:
unset key
set samp 600
set iso 600
set xr [0:5]
set yr [0:5]
e = 1.3
plot "++" u ($1 * rand(0)**e):($2 * rand(0)**e)\
  with points pt 7 lc "#fa990099" ps .5

Attention to Style
------------------

Gnuplot has a reputation in some quarters for producing unattractive
graphs. This is unfair, but due to the fact that gnuplot’s default
settings lead to output that, while utilitarian, can benefit from some
aesthetic tweaking. This example is simply a reminder that gnuplot
allows you to use any fonts on your system, and is extremely
customizable. A little attention to colors, the judicious use of
transparency, and careful selection of fonts, sizes, margins, and line
styles, can lead to visualizations limited in beauty only by your skill
in design.

If you don’t have one or both of the fonts that we call for in the
following script, the output from your machine will look different. This
example also serves to illustrate the `set style textbox` command. This
command is evolving, and may give different results, or not work at all,
depending on your particular gnuplot version. It defines a style for the
`boxed` variant of a text label, which, by default, creates a black
border. One of its quirks is that, to get a partially transparent fill,
as we show here, you must use the `opaque` keyword, and use an alpha
component for the `fillcolor`. If you say `transparent` rather than
`opaque`, that produces an box with no fill. Note also that the only way
to apply a detailed style to grid lines is with a user-defined
`linetype`.

In [ ]:
set border 3 lw 2 lc "grey40"
unset key
set samp 1000
set linetype 5 lc "grey80" lw 2 dt "-"
set grid lt 5
set xtics 4
set ytics 0.2
set tics font "Ubuntu Condensed, 16" tc "#555599"
set tics nomirror
set xr [-15 : 15]
set yr [-0.6 : 0.6]
set offsets 1
set style textbox opaque border rgb "#444444" lw 2 fc "#aa666666"\
                   margins 3, 2
set label "Airy function" font "Sawasdee, 18" at 3.5, 0.14 boxed
plot airy(x) lw 3 lc "#883433"